In [1]:
from netaddr import *
import pandas as pd
import sqlite3

conn = sqlite3.connect('/Users/fish/Downloads/ipDB.db')
c = conn.cursor()

ip_table = pd.read_sql('select 單位名稱, ID, 核發IP範圍 from ipTable', conn)
ip_table.head()

,單位名稱,ID,核發IP範圍
0,中華電信股份有限公司,HINET-NET,202.39.128.0 - 202.39.255.255
1,衛生福利部,HEALTHNET-NET,203.65.128.0 - 203.65.159.255
2,衛生福利部,HEALTHNET-NET,203.65.0.0 - 203.65.127.255
3,中華電信股份有限公司,HINET-NET,203.69.0.0 - 203.69.255.255
4,教育部,TANET-NET,203.68.0.0 - 203.68.255.255


In [2]:
company = []
ip_address = []

for com, IP in zip(ip_table['單位名稱'], ip_table['核發IP範圍']):
    ip_list = list(iter_iprange(IP.split(' - ')[0], IP.split(' - ')[1]))
    for com_ip in ip_list:
        company.append(com)
        ip_address.append(str(com_ip))

In [12]:
len(company)

33849344

In [13]:
len(ip_address)

33849344

In [20]:
taiwan_company_ip = pd.DataFrame(zip(company, ip_address), columns=['company', 'ip'])
taiwan_company_ip.to_sql('taiwan_company_ip', conn, if_exists='replace', index = False)

In [18]:
taiwan_company_ip = pd.read_sql('select * from taiwan_company_ip', conn)
taiwan_company_ip.head()

,company,ip
0,中華電信股份有限公司,202.39.128.0
1,中華電信股份有限公司,202.39.128.1
2,中華電信股份有限公司,202.39.128.2
3,中華電信股份有限公司,202.39.128.3
4,中華電信股份有限公司,202.39.128.4


In [16]:
not_bot_ip = pd.read_sql('select ip from dataset where bot is Null', conn)
ip_count = pd.DataFrame({'count' : not_bot_ip.groupby(['ip']).size()}).reset_index().sort_values('count', ascending=False)
ip_count.head()

,ip,count
34889,45.121.50.40,586647
14697,140.110.59.81,532844
36267,52.23.222.220,208726
14578,140.110.35.129,151791
14553,140.110.240.57,133628


In [15]:
ip_detail = pd.read_sql("select * from ip_detail", conn)
ip_detail.head()

,ip,ip_detail_column
0,223.200.90.67,None
1,117.19.224.80,117-19-224-80.dynamic.taiwanmobile.net
2,114.198.188.209,None
3,101.14.181.123,None
4,120.119.126.105,nat105.stu.edu.tw


In [14]:
no_detail_ip = ip_detail[ip_detail['ip_detail_column'].isnull()]
no_detail_ip.head()

,ip,ip_detail_column
0,223.200.90.67,None
2,114.198.188.209,None
3,101.14.181.123,None
6,196.64.118.155,None
10,163.24.47.84,None


In [19]:
df3 = no_detail_ip.join(taiwan_company_ip.set_index('ip'), on='ip')
df3

,ip,ip_detail_column,company
0,223.200.90.67,None,國家發展委員會
2,114.198.188.209,None,台固媒體股份有限公司
3,101.14.181.123,None,台灣大哥大股份有限公司
6,196.64.118.155,None,NaN
10,163.24.47.84,None,教育部
14,140.123.51.179,None,教育部
17,31.13.127.43,None,NaN
18,49.219.144.87,None,台灣固網股份有限公司
19,116.26.141.254,None,NaN
28,65.49.68.164,None,NaN


In [24]:
df3_drop = df3.drop(['ip_detail_column'], axis=1).dropna()
df3_drop

,ip,company
0,223.200.90.67,國家發展委員會
2,114.198.188.209,台固媒體股份有限公司
3,101.14.181.123,台灣大哥大股份有限公司
10,163.24.47.84,教育部
14,140.123.51.179,教育部
18,49.219.144.87,台灣固網股份有限公司
31,49.214.226.221,台灣固網股份有限公司
32,101.9.161.120,台灣大哥大股份有限公司
37,101.13.212.239,台灣大哥大股份有限公司
38,140.112.29.234,教育部


In [25]:
df3_drop.to_sql('ip_in_ip_detail', conn, if_exists='replace', index = False)

## 重新看 ip_detail 未查出來之 IP

In [27]:
ip_detail = pd.read_sql("select * from ip_detail", conn)
no_detail_ip = ip_detail[ip_detail['ip_detail_column'].isnull()]
no_detail_ip.head()

,ip,count,ip_detail_column
6,196.64.118.155,40,None
17,31.13.127.43,1,None
19,116.26.141.254,2,None
28,65.49.68.164,8,None
42,111.225.147.77,3,None


In [39]:
len(no_detail_ip)

6513

In [40]:
from ipwhois import IPWhois

In [44]:
IPWhois('140.110.240.1').lookup_rdap(depth=1)

{'nir': None,
 'asn_registry': 'apnic',
 'asn': '7539',
 'asn_cidr': '140.110.0.0/16',
 'asn_country_code': 'TW',
 'asn_date': '1992-12-18',
 'asn_description': 'TWAREN-TW National Center for High-performance Computing, TW',
 'query': '140.110.240.1',
 'network': {'handle': '140.109.0.0 - 140.111.255.255',
  'status': None,
  'remarks': [{'title': 'description',
    'description': 'imported inetnum object for MOEC',
    'links': None}],
  'notices': [{'title': 'Source',
    'description': 'Objects returned came from source\nAPNIC',
    'links': None},
   {'title': 'Terms and Conditions',
    'description': 'This is the APNIC WHOIS Database query service. The objects are in RDAP format.',
    'links': ['http://www.apnic.net/db/dbcopyright.html']}],
  'links': ['http://rdap.apnic.net/ip/140.109.0.0/14'],
  'events': [{'action': 'last changed',
    'timestamp': '2015-12-01T22:24:35Z',
    'actor': None}],
  'raw': None,
  'start_address': '140.109.0.0',
  'end_address': '140.111.255.255',

In [35]:
ip_description = []

for ip in no_detail_ip['ip']:
    try:
        description = IPWhois(ip).lookup_rdap(depth=1)['asn_description']
        ip_description.append(description)
    except:
        ip_description.append(None)

In [37]:
no_detail_ip['ip_detail_column'] = ip_description
no_detail_ip

/Users/fish/anaconda3/envs/ip_process/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ip,count,ip_detail_column
6,196.64.118.155,40,"MT-MPLS, MA"
17,31.13.127.43,1,"FACEBOOK - Facebook, Inc., US"
19,116.26.141.254,2,"CHINANET-BACKBONE No.31,Jin-rong Street, CN"
28,65.49.68.164,8,"HURRICANE - Hurricane Electric LLC, US"
42,111.225.147.77,3,"CHINANET-BACKBONE No.31,Jin-rong Street, CN"
46,111.225.148.130,6,"CHINANET-BACKBONE No.31,Jin-rong Street, CN"
49,173.252.87.2,3,"FACEBOOK - Facebook, Inc., US"
54,193.51.236.96,8,FR-RENATER Reseau National de telecommunicatio...
56,61.148.199.18,11,CHINA169-BJ China Unicom Beijing Province Netw...
72,110.249.202.63,7,CHINA169-BACKBONE CHINA UNICOM China169 Backbo...


In [38]:
no_detail_ip.to_sql('ipwhois', conn, if_exists='replace', index = False)